Install python packages which are reuqired in thei example. Here we are using "requirements.txt" file which contains a list of packages. 

In [ ]:
pip install -r requirements.txt --ignore-installed

Refrencing the libraries from the pythin packages

In [7]:
from typing import List
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from io import BytesIO
from huggingface_hub import hf_hub_download
from langchain.vectorstores import FAISS
import chainlit as cl
import io
import os
import PyPDF2
import urllib.request


2023-12-26 22:13:06 - Created default config file at /root/.chainlit/config.toml


In [5]:
DATA_PATH = 'localdata'
DB_FAISS_PATH = 'vectorstore'

In [10]:
# Parent Directory path 
parent_dir = "./"

# Create folder for localdata
directory = DATA_PATH
path = os.path.join(parent_dir, directory) 
if not os.path.exists(directory):
    os.mkdir(path) 
    
# PCreate folder for VectorStore
directory = DB_FAISS_PATH
path = os.path.join(parent_dir, directory) 
if not os.path.exists(directory):
    os.mkdir(path) 
    
urllib.request.urlretrieve("https://insights.sei.cmu.edu/documents/1287/2020_005_001_638589.pdf", "localdata/devsecops.pdf")

('localdata/devsecops.pdf', <http.client.HTTPMessage at 0x7fab44178070>)

In [11]:
custom_prompt_template = """Use the following context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [12]:
def hugging_embedding():
    
    # you can use a different embedding by replacing with model_name = sentence-transformers/all-MiniLM-L6-v2. system will download it during first execution
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5",
                                       model_kwargs={'device': 'cpu'},
                                       cache_folder="./models")
    return embeddings


In [13]:
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,
                                                   chunk_overlap=80)
    texts = text_splitter.split_documents(documents)
    
    # you can use a different embedding by replacing with model_name = sentence-transformers/all-MiniLM-L6-v2. system will download it during first execution
    embeddings = hugging_embedding()

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

In [14]:
def downloadmodel(model_id,model_filename,model_path_cache):
    model_path = hf_hub_download(             
                                 repo_id=model_id,             
                                 filename=model_filename,             
                                 resume_download=True,             
                                 cache_dir=model_path_cache,         
                                 )
    return model_path


In [15]:
def load_llm():

    #uncomment to allow the UI to download a different model during first execution
    mdpath = downloadmodel("TheBloke/Llama-2-7B-Chat-GGUF","llama-2-7b-chat.Q2_K.gguf","./models")

    # Load the locally downloaded model here
    llm = CTransformers(
        model = mdpath,
        model_type="llama",
        temperature = 0.4,
        repetition_penalty = 1.2,
        top_k = 50,
        context_length = 6000
    )
    return llm

In [16]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt


In [17]:
def retrieval_qa_chain(llm, prompt, db):     
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type='stuff',
                                           retriever=db.as_retriever(search_kwargs={'k': 2}),
                                           return_source_documents=True,
                                           chain_type_kwargs={'prompt': prompt}                                        
                                           )     
    return qa_chain

In [18]:
def qa_bot():
    # you can use a different embedding by replacing with model_name = sentence-transformers/all-MiniLM-L6-v2. system will download it during first execution
    embeddings = hugging_embedding()
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa


In [19]:
llm = load_llm()

llama-2-7b-chat.Q2_K.gguf:   0%|          | 0.00/2.83G [00:00<?, ?B/s]

In [18]:
llm("Q: Challenges of DevSecOps? A: ", max_tokens=32,  stop=["Q:", "\n"])

'1. surely you are thinking of DevOps, right? DevSecOps is a relatively new term that has emerged as the software development industry has evolved and matured. The challenges of DevSecOps are similar to those faced by organizations implementing DevOps, with a few additional considerations due to the security aspect. Here are some common challenges:'

In [20]:
create_vector_db()

2023-12-26 22:15:37 - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

2023-12-26 22:15:49 - Loading faiss with AVX2 support.
2023-12-26 22:15:49 - Successfully loaded faiss with AVX2 support.


In [21]:
chain = qa_bot()

2023-12-26 19:44:17 - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


In [61]:
query = "Challenges of DevSecOps"
chain(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'Challenges of DevSecOps',
 'result': 'Some challenges in incorporating security into a DevSecOps environment include:\n-Lack of awareness about the importance of security in DevOps\n-Insufficient resources and budget for security tools and training\n-Difficulty integrating security processes with existing workflows \n-Piecemeal approach to security, rather than holistic approach \n-Difficulty in measuring the effectiveness of security controls.',
 'source_documents': [Document(page_content='development in a DevOps \nenvironmentEnable Security \nAutomation\nAutomated security tools in the \nDevSecOps pipeline improves \noverall security by reducing \nvulnerabilities and security flaws \ndue to human error Automated Audit \nEvidence Collection  \nSecurity monitoring and notification \nsystems in DevSecOps creates an \nautomated audit trail throughout \nthe software development lifecycle,', metadata={'source': 'localdata/DevSecOps-Explained.pdf', 'page': 5}),
  Document(page_co

In [41]:
loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                     chunk_overlap=80)
texts = text_splitter.split_documents(documents)
len(texts)


25

In [42]:
embeddings = hugging_embedding()

db = FAISS.from_documents(texts, embeddings)

docs_and_scores = db.similarity_search_with_score("challenges of  devsecops")
docs_and_scores


2023-12-26 18:42:14 - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[(Document(page_content='require development \nknowledge\nDevSecOps just means \ncode scanning\nDevSecOps prevents \norganizations from meeting their \nbusiness objectives', metadata={'source': 'localdata/DevSecOps-Explained.pdf', 'page': 3}),
  0.37491366),
 (Document(page_content='Copyright © 2018 Deloitte Development LLC. All rights reserved. 4Perceived challenges and piece- meal integration often hinder organizations from realizing the value of \nincorporating security into DevOpsCommon myths and misconceptions\nDevSecOps is incompatible \nwith my compliance \nrequirementsDevSecOps is only “Security as Code” \nor Automation\nDevSecOps requires \nsignificant tool investmentDevSecOps requires developers to be security expertsSecurity team does not \nrequire development \nknowledge', metadata={'source': 'localdata/DevSecOps-Explained.pdf', 'page': 3}),
  0.4234941),
 (Document(page_content='operations for a productEstablish Policies \nand Procedures\nIntroducing DevSecOps \nspecific p